# Lorenz 96 problem setup for estimating spatially dependent forcing with neural network approximation, strong prior, and algorithm testing
By: Rebecca Gjini 

In [ ]:
#Import statements
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numba
from numba import jit, njit
from scipy.linalg import sqrtm
from scipy.optimize import least_squares
import main.EnsembleKalmanAlgorithms as EKA
import main.l96.nn_mL96_model as ml96
from cycler import cycler
import torch
import torch.nn as nn

#Plot preferences from palettable.colorbrewer.qualitative import Set1_9
from palettable.colorbrewer.qualitative import Dark2_8

#Plot Preferences 
plt.rcParams['figure.figsize'] = [11.0, 9.0] #size (w, h)
plt.rcParams['figure.dpi'] = 80
plt.rcParams['savefig.dpi'] = 400

plt.rcParams['font.size'] = 18 # controls default text sizes
plt.rcParams['legend.fontsize'] = 'large' # legend fontsize
plt.rcParams['figure.titlesize'] = 'large' # fontsize of the figure title  
plt.rcParams['axes.titlesize'] = 18     # fontsize of the axes title
plt.rcParams['axes.labelsize'] = 32   # fontsize of the x and y labels 35
plt.rcParams['xtick.labelsize'] = 20    # fontsize of the tick labels 17
plt.rcParams['ytick.labelsize'] = 20   # fontsize of the tick labels 17
plt.rcParams['axes.spines.right'] = False #makes right line in plot disappear
plt.rcParams['axes.spines.top'] = False #makes top line in plot disappear
plt.rcParams["font.family"] = "Times New Roman"

plt.rcParams['axes.prop_cycle'] = cycler(color=Dark2_8.mpl_colors)

In [ ]:
file_path = "main/l96/nn_mp_strong/"
#initialize random seed
np.random.seed(2)

In [ ]:
#Creating my sythetic data
#initalize model variables
nx = 100  #dimensions of parameter vector
gamma = 8 + 6*np.sin((4*np.pi*np.arange(0, nx, 1))/nx)  #forcing 

t = 0.005  #time step
T_long = 1000  #total time 

#beginning state
int_state = np.random.uniform(0,1,nx)

plt.figure()
plt.plot(np.arange(0,100,1), gamma)
plt.show()

In [ ]:
#Find the initial condition for my data
spin_up_array = ml96.runge_kutta_v(gamma, int_state, t, T_long)
#intital condition used for the data
x0 = spin_up_array[-1]
# np.savetxt(file_path + 'data/x0.txt', x0, delimiter = ',') 
# print(x0)

In [ ]:
# Neural net setup
# Initialize network
input_dim = 1
hidden_dim = 20
output_dim = 1
model = ml96.FeedforwardNN(input_dim, hidden_dim, output_dim)
# Generate test data
model_input = np.linspace(-5.0, 5.0, nx).reshape(-1, 1)
model_input_tensor = torch.tensor(model_input, dtype=torch.float32)


In [ ]:
#Creating my sythetic data
window = 50
T = 4  + window # total time units, with 4 unit cut off
ny = nx*2   #number of data points

weights = np.loadtxt("main/l96/nn_mp/" + "data/true_weights.txt", delimiter = ",")
np.savetxt(file_path + "data/true_weights.txt", weights, delimiter = ",")

model_out_y = ml96.G(weights, model, model_input_tensor, x0, t, T, np.zeros((nx, nx)))

# #Data covariance
# t_mult = 2*t
# T_cov = 100004
# RKgam = ml96.runge_kutta_v(gamma, x0, t, T_cov)[int(4/t_mult):] 
# gam = np.zeros((ny,int((T_cov - 4)/window)))
# for i in range(0, T_cov - 4, window):
#     ii = int(i/window)
#     gam[:nx,ii] = np.mean(RKgam[int(i/t_mult):int((i+window)/t_mult)], axis = 0)
#     gam[nx:,ii] = np.sqrt(np.var(RKgam[int(i/t_mult):int((i+window)/t_mult)], axis = 0, ddof = 1))

# R = 2.5*np.cov(gam) 

R = np.loadtxt("main/l96/nn_mp_constrained/" + 'data/R.txt', delimiter = ',')

R_sqrt_in = EKA.matrix_inv_sqrt(R)
R_sqrt = EKA.matrix_sqrt(R)

#Observations y
y = model_out_y


In [ ]:
plt.figure()
plt.imshow(R)
plt.colorbar()
plt.show()

In [ ]:
weights_len = len(weights)

# One sample plus noise prior
mu = np.loadtxt(file_path + f"sinusoid_samples/sample_2.txt")
B = (0.1**2)*np.identity(weights_len)
B_sqrt = EKA.matrix_sqrt(B)
B_sqrt_in = EKA.matrix_inv_sqrt(B)

# Solving for initial condition perturbation covariance
covT = 2000  #time to simulate to calculate a covariance matrix of the system
cov_solve = ml96.runge_kutta_v(gamma, x0, t, covT)
ic_cov = 0.1*np.cov(cov_solve.T)
ic_cov_sqrt =  EKA.matrix_sqrt(ic_cov)

# Save problem data, prior, and covarainces 
np.savetxt(file_path + 'data/y.txt', y, delimiter = ',') 
# np.savetxt(file_path + 'data/R.txt', R, delimiter = ',')
# np.savetxt(file_path + 'data/mu.txt', mu, delimiter = ',')
# np.savetxt(file_path + 'data/B.txt', B, delimiter = ',')
# np.savetxt(file_path + 'data/ic_cov_sqrt.txt', ic_cov_sqrt, delimiter = ',')

In [ ]:
print(mu.shape)
plt.figure()
plt.imshow(B)
plt.colorbar()
plt.show()

In [ ]:
# ml96.load_weights(model, B_samples[:,0]) 
ml96.load_weights(model, mu + (B_sqrt)@np.random.normal(0, 1, size = (mu.shape)))
prior_sample = (8 + 6*model(model_input_tensor).detach().numpy())

plt.figure()
plt.plot(model_input, prior_sample)
plt.show()

In [ ]:
def loss(ws, models, model_inputs, x_0s, ts, Ts, ic_cov_sqrts, ys, R_sqrt_ins, mus, B_sqrt_ins):
    data_model = 0.5*np.linalg.norm(R_sqrt_ins@(ys - ml96.G(ws, models, model_inputs, x_0s
                                                               ,ts, Ts, ic_cov_sqrts)))**2 
                                                               #np.sqrt(2*()/len(y))
    prior = 0.5*np.linalg.norm(B_sqrt_ins@(mus -ws))**2
    return data_model + prior

#y = model_out_y + R_sqrt@np.random.normal(0, 1, size = (len(model_out_y)))

truth = loss(weights, model, model_input_tensor, x0, t, T, 0*ic_cov_sqrt, y, R_sqrt_in, mu, B_sqrt_in)

#Plotting in different planes of the cost function
indices = np.arange(0, 61, 10)
for ii in indices: 
    weight_vals = np.arange(-5, 5, 0.1) + weights[ii]
    cost_function = np.zeros(len(weight_vals))
    for jj in range(0, len(weight_vals)): 
        weights_j = np.concatenate((weights[:ii], [weight_vals[jj]], weights[ii + 1:]))
        cost_function[jj] = loss(weights_j, model, model_input_tensor, x0, t, T, ic_cov_sqrt, y, 
                                 R_sqrt_in, mu, B_sqrt_in)
        
    plt.figure()
    plt.plot(weight_vals, cost_function, linewidth = 3)
    #plt.title('Cost function of rho')
    plt.ylabel('Loss')
    plt.xlabel('Weight of bias number = %d' %ii)
    plt.show()


In [ ]:
#Intitializing EKI ensemble
K = 80       #number of ensemble members

max_runs = 20   #set a maximum number of runs 

N_t = weights_len         #we estimate the forcing

# Initialize network
model = ml96.FeedforwardNN(input_dim, hidden_dim, output_dim)
# Generate test data
model_input_tensor = torch.tensor(model_input, dtype=torch.float32)

u = np.random.normal(0, 1, size = (N_t,K)) # B_sqrt_in@(B_samples[:,:K] - mu[:, np.newaxis]) 
print(u.shape) 

In [ ]:
#TEKI Test 
teki_u, teki_f, _ = EKA.TEKI(ml96.G, u, (model, model_input_tensor, x0, t, T, ic_cov_sqrt), 
                          y, R, mu, B, min_rmse = 1.0, method = 'all', 
                             tol_x = 1e-8, tol_f = 1e-8, max_iter = max_runs)
print(teki_f)
ft = ml96.G(np.mean(teki_u, axis = 1), model, model_input_tensor, x0, t, T, ic_cov_sqrt)
np.sqrt((np.linalg.norm(R_sqrt_in@(y - ft))**2)/len(y)) 

In [ ]:
ml96.load_weights(model, np.mean(teki_u, axis = 1))
teki_prediction = model(model_input_tensor).detach().numpy()

plt.figure()
plt.plot(8 + 6*teki_prediction)
plt.show()

In [ ]:
#UKI Test
uki_u, uki_f, _ = EKA.UKI(ml96.G, (model, model_input_tensor, x0, t, T, ic_cov_sqrt), 
                         y, R, mu, B, a = 1.0, min_rmse = 1.0,  method = 'all', 
                       tol_x = 1e-8, tol_f = 1e-8, max_iter = 20)
print(uki_f)
fu = ml96.G(np.mean(uki_u, axis = 1), model, model_input_tensor, x0, t, T, ic_cov_sqrt)
np.sqrt((np.linalg.norm(R_sqrt_in@(y - fu))**2)/len(y))

In [ ]:
etki_u, etki_f, exit = EKA.ETKI(ml96.G, u, (model, model_input_tensor, x0, t, T, ic_cov_sqrt), 
                          y, R, mu, B, min_rmse = 1.0, method = 'all', 
                          tol_x = 1e-8, tol_f = 1e-8, max_iter = max_runs)
print(exit)
print(etki_f)
fe = ml96.G(np.mean(etki_u, axis = 1), model, model_input_tensor, x0, t, T, ic_cov_sqrt)
np.sqrt((np.linalg.norm(R_sqrt_in@(y - fe))**2)/len(y))

In [ ]:
iekf_u, iekf_f, exit = EKA.IEKF_stable(ml96.G, u, (model, model_input_tensor, x0, t, T, ic_cov_sqrt), 
                          y, R, mu, B, alpha = 1.0, min_rmse = 1.0, 
                           method = 'all', tol_x = 1e-4, tol_f = 1e-4, max_iter = 100)
print(iekf_f)
print(exit)
fi = ml96.G(np.mean(iekf_u, axis = 1), model, model_input_tensor, x0, t, T, ic_cov_sqrt)
np.sqrt((np.linalg.norm(R_sqrt_in@(y - fi))**2)/len(y))

In [ ]:
solution = least_squares(ml96.r, u[:,0], args=(model, model_input_tensor, x0, t, T, ic_cov_sqrt, y, R_sqrt_in, mu, B_sqrt), method = 'lm', 
                         xtol = 1e-8, ftol=1e-08)
print(solution.nfev)
print(B_sqrt@solution.x + mu)
print(solution.status)

In [ ]:
f = ml96.G(B_sqrt@solution.x + mu, model, model_input_tensor, x0, t, T, ic_cov_sqrt)
np.sqrt((np.linalg.norm(R_sqrt_in@(y - f))**2)/len(y))

In [ ]:
ml96.load_weights(model, np.mean(teki_u, axis = 1))
teki_prediction = 8 + 6*model(model_input_tensor).detach().numpy()
ml96.load_weights(model, np.mean(uki_u, axis = 1))
uki_prediction = 8 + 6*model(model_input_tensor).detach().numpy()
ml96.load_weights(model, np.mean(etki_u, axis = 1))
etki_prediction = 8 + 6*model(model_input_tensor).detach().numpy()
ml96.load_weights(model, np.mean(iekf_u, axis = 1))
iekf_prediction = 8 + 6*model(model_input_tensor).detach().numpy()
# ml96.load_weights(model, B_sqrt@solution.x + mu)
# lm_prediction = 8 + 6*model(model_input_tensor).detach().numpy()
FI = np.arange(0,nx,1)
plt.figure()
plt.plot(FI, gamma, label = 'true forcing', c = 'black', linewidth = 2)
plt.plot(FI, teki_prediction, label = 'teki')
plt.plot(FI, uki_prediction, label = 'uki')
plt.plot(FI, etki_prediction, label = 'etki')
plt.plot(FI, iekf_prediction, label = 'iekf')
# plt.plot(FI, lm_prediction, label = 'lmfd')
plt.xlabel('Data point')
plt.ylabel('Model output')
plt.legend()
plt.show()

In [ ]:
# Plot of data vs predicted data
FI = np.arange(0,nx,1)
# Means
plt.figure()
plt.errorbar(FI, y[:100], label = 'true forcing', 
             markerfacecolor='none', markeredgecolor='black',
             c = 'black', yerr=2*np.sqrt(np.diag(R)[:100]), 
             fmt='o', capsize=6, capthick=1, elinewidth=2,
             zorder=1, markersize = 10)
plt.plot(FI, ft[:100], label = 'teki', linewidth = 3)
plt.plot(FI, fu[:100], label = 'uki', linewidth = 3)
plt.plot(FI, fe[:100], label = 'etki', linewidth = 3)
plt.plot(FI, fi[:100], label = 'iekf', linewidth = 3)
plt.xlabel('Index number')
plt.ylabel('Means')
plt.legend()
plt.show()

#Standard deviations
plt.figure()
plt.errorbar(FI, y[100:], label = 'true forcing',
            markerfacecolor='none', markeredgecolor='black',  
            yerr=2*np.sqrt(np.diag(R)[100:]), c = 'black',
            fmt='o', capsize=6, capthick=1, elinewidth=2,
            zorder=1, markersize = 10)
plt.plot(FI, ft[100:], label = 'teki', linewidth = 3)
plt.plot(FI, fu[100:], label = 'uki', linewidth = 3)
plt.plot(FI, fe[100:], label = 'etki', linewidth = 3)
plt.plot(FI, fi[100:], label = 'iekf', linewidth = 3)
plt.xlabel('Index number')
plt.ylabel('Standard deviations')
plt.legend()
plt.show()

